# Pilot 3


In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### import data

In [ ]:
base_directory = '../data' # directory where results are

output_path = base_directory + '/compiled_data.csv' # output csv of compiled data (to later merge with exported demographic data) and csv of comments

prolific_df_path = base_directory + '/prolific_demographic_data.csv' # path to exported prolific demographic data

def extract_first_value(df, column_name):
    """
    Extracts the first non-empty value from a DataFrame column
    """
    if column_name not in df.columns:
        return np.nan

    value_series = df[df[column_name].notna() & (df[column_name] != '')]

    if not value_series.empty:
        return value_series[column_name].iloc[0]

    return np.nan

# data was downloaded using JATOS results archive
all_participants_data = []
search_pattern = os.path.join(base_directory, '**', '*.txt') # find all participant data files
data_files = glob.glob(search_pattern, recursive=True)

for file_path in data_files:
    temp_df = pd.read_csv(file_path)

    participant_vars = {
        'success': np.nan,
        'subject_id': np.nan,
        'study_id': np.nan,
        'session_id': np.nan,
        'audio_check_passed': np.nan,
        'audio_checks': np.nan,
        'condition': np.nan,
        'comprehension_failures': np.nan,
        'collision_check': np.nan,
        'heard_sound': np.nan,
        'worker_comments': "",
        'cork_guess': np.nan,
        'guess': np.nan,
        'cork_confidence': np.nan,
        'confidence': np.nan
    }

    # extract ids
    participant_vars['subject_id'] = extract_first_value(temp_df, 'PROLIFIC_PID')
    participant_vars['study_id'] = extract_first_value(temp_df, 'STUDY_ID')
    participant_vars['session_id'] = extract_first_value(temp_df, 'SESSION_ID')

    # extract response to 2AFC
    guess_rows = temp_df[temp_df['guess'].notna()]
    if not guess_rows.empty:
        participant_vars['cork_guess'] = guess_rows['guess'].iloc[0] # guess for cork trial
        participant_vars['guess'] = guess_rows['guess'].iloc[1] # guess for plastic trial

    # extract confidence
    slider_rows = temp_df[temp_df['trial_type'] == 'html-slider-response']
    if not slider_rows.empty:
        participant_vars['cork_confidence'] = slider_rows['confidence'].iloc[0]
        participant_vars['confidence'] = slider_rows['confidence'].iloc[1]

    # check for hearing collision sound
    sound_row = temp_df[temp_df['heard_sound'].notna() & (temp_df['heard_sound'] != '')]
    if not sound_row.empty:
        participant_vars['heard_sound'] = sound_row['heard_sound'].iloc[0]

    # collision check and comments
    for response_str in temp_df['response'].dropna():
        try:
            response_json = json.loads(response_str)
            if 'collision_check' in response_json:
                participant_vars['collision_check'] = response_json['collision_check']
            if 'worker_comments' in response_json:
                participant_vars['worker_comments'] = response_json['worker_comments']
        except (json.JSONDecodeError, TypeError):
            continue


    # fallback loop for remaining single-value keys
    for key in participant_vars.keys():
        if pd.isna(participant_vars[key]) or participant_vars[key] == "":
            participant_vars[key] = extract_first_value(temp_df, key)

    all_participants_data.append(participant_vars)


if all_participants_data:
    final_df = pd.DataFrame(all_participants_data)

    column_order = [
        'subject_id',
        'study_id',
        'session_id',
        'condition',
        'success',
        'audio_check_passed',
        'audio_checks',
        'comprehension_failures',
        'cork_guess',
        'cork_confidence',
        'guess',
        'confidence',
        'collision_check',
        'heard_sound',
        'worker_comments',
    ]

    final_df = final_df[column_order]

    final_df.to_csv(output_path, index=False)

    print(f"{final_df.shape[0]} participants data written to {output_path}")

    # output csv for comments
    comments_df = final_df[final_df['worker_comments'].notna() & (final_df['worker_comments'] != '')].copy()

    if not comments_df.empty:
        comments_output_path = output_path.replace('.csv', '_with_comments.csv')
        comments_df.to_csv(comments_output_path, index=False)

    else:
        print("\n No comments")
else:
    print("\nNo data was processed.")


##### merge dataframe with demographic data #####
# not particularly relevant for this analysis but useful to know which participants data are missing

participant_df = pd.read_csv(output_path)
prolific_df = pd.read_csv(prolific_df_path)

participant_ids = set(participant_df['subject_id'].dropna())
prolific_ids = set(prolific_df['Participant id'].dropna())
unmatched_ids = participant_ids.difference(prolific_ids)

if unmatched_ids:
    print("--- Unmatched Participant IDs (Data -> Prolific) ---")
    print("The following IDs from were NOT found in the Prolific export:")
    for participant_id in sorted(list(unmatched_ids)):
        print(f"- {participant_id}")
else:
    print("--- ID Check (Data -> Prolific) ---")
    print("All participant IDs matched with the Prolific export.")

# filter for completed participants
completed_prolific_df = prolific_df[prolific_df['Completed at'].notna()]
completed_prolific_ids = set(completed_prolific_df['Participant id'].dropna())

# find mismatches
missing_from_data_ids = completed_prolific_ids.difference(participant_ids)

# Print the results
if missing_from_data_ids:
    print("--- Missing Participant Data (Prolific -> Data) ---")
    print("Found in Prolific but not in Data:")
    for participant_id in sorted(list(missing_from_data_ids)):
        print(f"- {participant_id}")
else:
    print("--- Data Completeness Check (Prolific -> Data) ---")
    print("All participant IDs matched with the Data")


# merge data
merged_df = pd.merge(
    left=participant_df,
    right=prolific_df,
    left_on='subject_id',
    right_on='Participant id',
    how='inner'
)

# save to csv (OPTIONAL)
output_filename = 'final_merged_data.csv'
full_output_path = os.path.join(os.path.dirname(output_path), output_filename)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
merged_df.to_csv(full_output_path, index=False)
print(f"Merged data file saved to: {full_output_path}")



### clean data

In [ ]:
# full dataframe
df = merged_df.copy()


df_clean = df[
    (df['audio_check_passed'] == True) & # pass audio check
    (df['comprehension_failures'] <= 2) & # pass comprehension check
    # remove did not hear sound in LCE and heard sound in SCE
    ((df['condition'] != 'LCE') | (df['heard_sound'] != 'no')) &
    ((df['condition'] != 'SCE') | (df['heard_sound'] != 'yes'))
]

df_clean['when'] = np.where(
    (df_clean['condition'] == 'SCE') | (df_clean['condition'] == 'LCE'),
    'early',
    'late'
)

df_clean['sound'] = np.where(
    (df_clean['condition'] == 'LCE') | (df_clean['condition'] == 'LCL'), # LCE and LCL are loud
    'loud',
    'silent'
)

print(f"Number of participants after cleaning: {len(df_clean)}")
print(f"SCE: {len(df_clean[df_clean['condition']=='SCE'])}")
print(f"LCE: {len(df_clean[df_clean['condition']=='LCE'])}")

### convert confidence to common confidence scale

In [ ]:
'''
confidence_scaled calculation
confidence: c

if guess was 2 discs:
    -c/2
if guess was 4 discs:
    c/2
i.e. positive means confident there were 4 discs, negative means confident there were 2 discs
'''
# confidence for plastic discs
df_clean['confidence_scaled'] = np.where(
    df_clean['guess'] == 2.0,          # 2 discs
    -df_clean['confidence'] / 2,       #
    df_clean['confidence'] / 2         # Value if false
)

# confidence for cork discs
df_clean['cork_confidence_scaled'] = np.where(
    df_clean['cork_guess'] == 2.0,          # Condition: if 'guess' is 2
    -df_clean['cork_confidence'] / 2,       # Value if true
    df_clean['cork_confidence'] / 2         # Value if false
)

### summary statistics

In [ ]:
# take the difference between confidence responses
df_clean['diff'] = df_clean['confidence_scaled'] - df_clean['cork_confidence_scaled'] # plastic - cork confidence
# positive: more sure that there were four discs, negative: more sure that there were two discs

# SCE
plastic_sce = df_clean[df_clean['condition'] == 'SCE']['confidence_scaled']
cork_sce = df_clean[df_clean['condition'] == 'SCE']['cork_confidence_scaled']
diff_sce = df_clean[df_clean['condition'] == 'SCE']['diff']

# LCE
plastic_lce = df_clean[df_clean['condition'] == 'LCE']['confidence_scaled']
cork_lce = df_clean[df_clean['condition'] == 'LCE']['cork_confidence_scaled']
diff_lce = df_clean[df_clean['condition'] == 'LCE']['diff']

print("Summary statistics of Scaled Confidence: \n")
print(f'plastic SCE:\n{plastic_sce.describe().round(2)}')
print(f'\nplastic LCE:\n{plastic_lce.describe().round(2)}')
print(f'\ncork SCE:\n{cork_sce.describe().round(2)}')
print(f'\ncork LCE:\n{cork_lce.describe().round(2)}')

print('\n------------------------------------\n')
print('Mean and Std of Delta Confidence: \n')
print(f'delta conf SCE:\n{diff_sce.describe().round(2)}')
print(f'\ndelta conf LCE:\n{diff_lce.describe().round(2)}')

### Plots

#### Figure 1: Distribution of scaled confidence by condition and material

In [ ]:
# change format of df for plotting
df_long = pd.melt(
    df_clean,
    id_vars=['subject_id', 'condition'],
    value_vars=['cork_confidence_scaled', 'confidence_scaled'],
    var_name='trial_type',
    value_name='scaled_confidence'
)

label_map = {
    'cork_confidence_scaled': 'cork',
    'confidence_scaled': 'plastic'
}
df_long['x_category'] = df_long['condition'] + ' ' + df_long['trial_type'].map(label_map)

# plotting figure
plt.figure(figsize=(12, 7))

plot_order = [
    'SCE cork',
    'SCE plastic',
    'LCE cork',
    'LCE plastic'
]

custom_palette = {
    'cork_confidence_scaled': 'forestgreen',
    'confidence_scaled': 'firebrick'
}

# box plot of confidence distribution according to condition and material
sns.boxplot(
    data=df_long,
    x='x_category',
    y='scaled_confidence',
    hue='trial_type',
    order=plot_order,
    palette=custom_palette,
    fliersize=0,
    dodge=False
)

# draw line in points within same subject to show directionality
sns.lineplot(
    data=df_long,
    x='x_category',
    y='scaled_confidence',
    units='subject_id',
    estimator=None,
    color='grey',
    alpha=0.4,
    legend=False,
    marker='o'
)

# for the legend
green_patch = mpatches.Patch(color='forestgreen', label='cork')
red_patch = mpatches.Patch(color='firebrick', label='plastic')

plt.legend(handles=[green_patch, red_patch], title="Disc Material")
plt.title('Distribution of Scaled Confidence by Condition and Material', fontsize=16)
plt.xlabel('Condition and material', fontsize=12)
plt.ylabel('Scaled Confidence (← Sure 2  |  Sure 4 →)', fontsize=12)
plt.ylim(-50, 50)
plt.yticks(np.arange(-50, 51, 10))
plt.xticks(rotation=10)
plt.axhline(0, color='grey', linestyle='--', linewidth=1)
plt.tight_layout()

plt.show()

#### Figure 2: Delta confidence distribution

In [ ]:
plt.figure(figsize=(10, 6))

# simple violin plot of condition and delta confidence
sns.violinplot(
    data=df_clean,
    x='condition',
    y='diff',
    inner='quart',
    color=".8"
)

# jitter points
sns.stripplot(
    data=df_clean,
    x='condition',
    y='diff',
    jitter=True,
    color='black',
    size=3,
    alpha=0.5
)

plt.title('Delta Confidence', fontsize=16)
plt.xlabel('Condition', fontsize=12)
plt.ylabel('← More sure 2 plastic discs  |  More sure 4 plastic discs →', fontsize=12)
plt.ylim(-100, 100)
plt.yticks(np.arange(-100, 101, 10))
plt.tight_layout()

#### Distribution of collision by condition

In [ ]:
plt.figure(figsize=(10, 6))

category_order = [
    'Exactly zero',
    'Exactly one',
    'At least one, but possibly more',
    'Definitely more than one'
]


sns.countplot(
    data=df_clean,
    x='collision_check',
    hue='condition',
    order=category_order
)

plt.title('Distribution of Collision Guesses by Condition', fontsize=16)
plt.xlabel('Collision Guess', fontsize=12)
plt.ylabel('Number of Participants', fontsize=12)
plt.xticks(rotation=25, ha='right')
plt.legend(title='Condition', loc='upper right')
plt.tight_layout()

#### Distribution of confidence by collision guess (Plastic)

In [ ]:
plt.figure(figsize=(10, 6))

category_order = [
    'Exactly zero',
    'Exactly one',
    'At least one, but possibly more',
    'Definitely more than one'
]


sns.boxplot(
    data=df_clean,
    x='collision_check',
    y = 'confidence_scaled',
    hue='condition',
    order=category_order
)


plt.title('Distribution of Collision Guesses by Condition (Plastic)', fontsize=16)
plt.xlabel('Collision Guess', fontsize=12)
plt.ylabel('Scaled Confidence (← Sure 2  |  Sure 4 →)', fontsize=12)
plt.xticks(rotation=25, ha='right')
plt.legend(title='Condition', loc='upper right')
plt.ylim(-50, 50)
plt.yticks(np.arange(-50, 51, 10))
plt.tight_layout()

### analysis

#### function for cohen's d

In [ ]:
def cohen_d_independent(x, y):
    """

    Args:
        x: data of one group (array-like)
        y: data of another group (array-like)
        (samples independent of one another)

    Returns:
        float: effect size, cohen's d for independent samples

    """
    len_x, len_y = len(x), len(y)
    mean_x, mean_y = np.mean(x), np.mean(y)
    std_x, std_y = np.std(x, ddof=1), np.std(y, ddof=1)

    pooled_std = np.sqrt(((len_x - 1) * std_x ** 2 + (len_y - 1) * std_y ** 2) / (len_x + len_y - 2))

    if pooled_std == 0:
        return np.inf

    d = (mean_x - mean_y) / pooled_std
    return d

def cohen_d_paired(x, y):
    """

    Args:
        x: first sample (array-like)
        y: second sample (array-like)

    Returns:
        float: effect size, cohen's d for paired samples
    """

    differences = np.array(x) - np.array(y)

    mean_diff = np.mean(differences)

    std_diff = np.std(differences, ddof=1)

    if std_diff == 0:
        return np.inf

    d = mean_diff / std_diff
    return d

#### Hypothesis 1 - 3

##### Two-way ANOVA on $C_{plastic}$ with factors sound (silent / loud) and collision time (early / late)


In [ ]:
model = ols('confidence_scaled ~ C(sound) + C(when)', data=df_clean).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print('Two-way ANOVA on plastic confidence')
print(anova_table)

##### Two-way ANOVA on $\Delta C$ with factors sound (silent / loud) and collision time (early / late)

In [ ]:
model = ols('diff ~ C(sound) + C(when)', data=df_clean).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print('Two-way ANOVA on plastic confidence')
print(anova_table)

##### $C_{plastic}$ SCE vs LCE, $\Delta C$ SCE versus LCE, $C_{plastic}$ SCL vs LCL, $\Delta C$ SCL versus LCL (depending on the results of the interaction effects)

In [ ]:
# E.g.
t_statistic, p_value = stats.ttest_ind(plastic_sce, plastic_lce, equal_var=False)

print(f"Independent samples T-test comparing confidence for plastic discs between SCE and LCE conditions.")
print(f"T-statistic: {t_statistic:.5f}")
print(f"P-value: {p_value:.5f}")
print(f"Cohen's d: {cohen_d_independent(plastic_sce, plastic_lce):.5f}")

#### Hypothesis 4

##### $\chi$-squared test for collision check responses

In [ ]:
df_clean = df_clean[df_clean['collision_check'] != 'Exactly zero']

df_clean['collision_check_merged'] = df_clean['collision_check'].replace({
    'At least one, but possibly more': 'More than one',
    'Definitely more than one': 'More than one'
})
    
contingency_table = pd.crosstab(df_clean['condition'], df_clean['collision_check_merged'])

chi2_stat, p_value, dof, expected_freq = stats.chi2_contingency(contingency_table)

print("Chi-Squared Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat:.5f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p_value:.5f}")

##### Multiple linear regression model of $C_{plastic}$ 

In [ ]:
formula = 'confidence_scaled ~ C(collision_check_merged) + C(condition) + C(collision_check_merged):C(condition)'

model = smf.ols(formula, data=df_clean).fit()


print("Multiple Linear Regression Results")
print("="*35)
print(f"Dependent Variable: C_plastic")
print(f"Independent Variables: Perceived Collisions, Condition, Interaction")
print("-" * 35 + "\n")
print(model.summary())